In [ ]:
%load_ext autoreload
%autoreload 2

In [9]:
import sys
sys.path.append('/Users/christinedekock/wp_internship/collaboration/')

import pandas as pd
from dateutil import parser
import datetime
import numpy as np
import json
import matplotlib.pylab as plt

from utils import get_edits_pre_tag
from features.politeness import *

In [10]:
revisions = pd.read_csv('/Users/christinedekock/wp_internship/data/formatted_data/autobiography.csv')

/Users/christinedekock/opt/anaconda3/envs/ml_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
revisions['event_timestamp'] = pd.to_datetime(revisions['event_timestamp'])
articles = revisions[revisions.page_namespace==0]

In [43]:
articles['event_user_id'] = articles['event_user_id'].astype(str)
articles['revision_id'] = articles['revision_id'].astype(str)

/Users/christinedekock/opt/anaconda3/envs/ml_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/christinedekock/opt/anaconda3/envs/ml_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
utts = articles.to_dict('records')


 17%|█▋        | 105249/627915 [00:29<01:03, 8282.90it/s]

In [45]:
users = [utt['event_user_id'] for utt in utts]

In [46]:
def reconstruct_corpus(dataset):
    users = [utt['event_user_id'] for utt in dataset] 
    users = np.unique(users)
    users_dict = {user: User(name=user) for user in users}

    utterances = []

    for utt in tqdm(dataset):
        user = users_dict[utt['event_user_id']] if utt['event_user_id'] is not None else users_dict['none']
        utterances.append(Utterance(id=utt['revision_id'], user=user, text=utt['event_comment\n']))

    corpus = Corpus(utterances=utterances)

    return corpus

In [47]:
corpus = reconstruct_corpus(utts)



  0%|          | 0/627915 [00:00<?, ?it/s]

  6%|▌         | 35161/627915 [00:00<00:01, 351609.50it/s]

 11%|█         | 68425/627915 [00:00<00:01, 345694.37it/s]

 16%|█▋        | 103594/627915 [00:00<00:01, 347471.34it/s]

 22%|██▏       | 137668/627915 [00:00<00:01, 345421.27it/s]

 27%|██▋       | 166877/627915 [00:00<00:01, 327482.00it/s]

 32%|███▏      | 199550/627915 [00:00<00:01, 327254.02it/s]

 37%|███▋      | 232947/627915 [00:00<00:01, 329239.44it/s]

 43%|████▎     | 266915/627915 [00:00<00:01, 332303.45it/s]

 48%|████▊     | 299824/627915 [00:00<00:00, 331330.80it/s]

 53%|█████▎    | 334878/627915 [00:01<00:00, 336378.96it/s]

 59%|█████▉    | 371649/627915 [00:01<00:00, 345202.35it/s]

 65%|██████▍   | 407187/627915 [00:01<00:00, 348193.76it/s]

 70%|███████   | 442164/627915 [00:01<00:00, 348655.54it/s]

 76%|███████▌  | 478355/627915 [00:01<00:00, 352345.77it/s]

 82%|████████▏ | 513398/627915 [00:03<00:01, 62278.25it/s] 

 88%|████████▊ | 549762/627915 [00:03<00:

In [48]:
parser = TextParser(verbosity=0)
parsed_corpus = parser.transform(corpus)

In [49]:
ps = PolitenessStrategies(verbose=False)
awry_corpus = ps.transform(parsed_corpus, markers=True)

In [50]:
feat_dict = format_politeness_features(awry_corpus)



  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:07<00:00,  7.35s/it]


In [51]:
politeness_dataset = []

for d in dataset:
    conv_id = d[0]['conv_id']
    features = feat_dict[conv_id]
    politeness_dataset.append(features)

NameError: name 'dataset' is not defined